In [ ]:
nvidia -smi

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Human-Path-Prediction-master/ynet

/content/drive/MyDrive/Human-Path-Prediction-master/ynet


In [ ]:
!ls

 config				    requirements.txt
 data				    saved_models
'evaluate_inD_longterm (1).ipynb'   segmentation_models
 evaluate_inD_longterm.ipynb	    test.py
 evaluate_SDD_longterm.ipynb	    train_inD_longterm.ipynb
 evaluate_SDD_trajnet.ipynb	    train.py
 images				    train_SDD_longterm.ipynb
 model.py			    train_SDD_trajnet.ipynb
 pretrained_models		    utils
 __pycache__			    w_andb
 README.md			    weights_and_biases.py


In [ ]:
!pip install wandb

In [ ]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet, YNetEncoder
import weights_and_biases as wandb

In [ ]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [ ]:
#@title Default title text
CONFIG_FILE_PATH = 'config/sdd_longterm.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'sdd_longterm'  # arbitrary name for this experiment
DATASET_NAME = 'sdd'

TRAIN_DATA_PATH = 'data/SDD/train_longterm.pkl'
TRAIN_IMAGE_PATH = 'data/SDD/train'
VAL_DATA_PATH = 'data/SDD/test_longterm.pkl'
VAL_IMAGE_PATH = 'data/SDD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 4

#### Load config file and print hyperparameters

In [ ]:
pip install -r requirements.txt

In [ ]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.002,
 'resize': 0.25,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.8,
 'unfreeze': 100,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [14, 29]}

#### Load preprocessed Data

In [ ]:
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

In [ ]:
df_train.head()

,trackId,frame,x,y,sceneId,metaId
0,2,6881,17.0,893.5,bookstore_0,0
1,2,6911,31.0,904.0,bookstore_0,0
2,2,6941,63.0,910.5,bookstore_0,0
3,2,6971,98.5,917.5,bookstore_0,0
4,2,7001,134.0,919.5,bookstore_0,0


#### Initiate model and load pretrained weights

In [ ]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [ ]:
wandb.init_wandb(params.copy(), model.model)

wandb: Currently logged in as: pradipto611 (use `wandb login --relogin` to force relogin)


#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 184/184 [00:00<00:00, 535.68it/s]
/content/drive/My Drive/Human-Path-Prediction-master/ynet/utils/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training
Epoch 0: 
Val ADE: 223.01263427734375 
Val FDE: 155.7736053466797
Best Epoch 0: 
Val ADE: 223.01263427734375 
Val FDE: 155.7736053466797


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   0%|          | 1/300 [27:37<137:41:11, 1657.76s/it]

Saved model to: saved_models/Ynet.pt
Epoch 1: 
Val ADE: 201.48619079589844 
Val FDE: 151.62229919433594
Best Epoch 1: 
Val ADE: 201.48619079589844 
Val FDE: 151.62229919433594


...

Saved model to: saved_models/Ynet.pt
Epoch 2: 
Val ADE: 186.65863037109375 
Val FDE: 137.8663330078125
Best Epoch 2: 
Val ADE: 186.65863037109375 
Val FDE: 137.8663330078125


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   1%|          | 3/300 [1:22:57<136:51:32, 1658.90s/it]

Saved model to: saved_models/Ynet.pt
Epoch 3: 
Val ADE: 175.9214630126953 
Val FDE: 139.49879455566406
Best Epoch 3: 
Val ADE: 175.9214630126953 
Val FDE: 139.49879455566406


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   1%|▏         | 4/300 [1:50:35<136:23:02, 1658.73s/it]

Saved model to: saved_models/Ynet.pt
